In [ ]:
import json
import gzip
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns 
import matplotlib.pyplot as plt
import glob
import zipfile
%matplotlib inline
fig = plt.figure()


In [4]:
# need to specify name and path and avoid duplicate downloads
!wget -O matric_demo_sidcaad2ce9d34900c599a16c2d8b2ba8f1_discoveries_parsed_csv_21.zip http://edx.netl.doe.gov/resource/dfd1938a-c890-442b-8341-c59d8fe6ee4d/download

URL transformed to HTTPS due to an HSTS policy
--2023-03-29 13:01:13--  https://edx.netl.doe.gov/resource/dfd1938a-c890-442b-8341-c59d8fe6ee4d/download
Resolving edx.netl.doe.gov (edx.netl.doe.gov)... 204.154.136.32
Connecting to edx.netl.doe.gov (edx.netl.doe.gov)|204.154.136.32|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://edx.netl.doe.gov/user/login?came_from=http%3A%2F%2Fedx.netl.doe.gov%2Fresource%2Fdfd1938a-c890-442b-8341-c59d8fe6ee4d%2Fdownload [following]
--2023-03-29 13:01:14--  https://edx.netl.doe.gov/user/login?came_from=http%3A%2F%2Fedx.netl.doe.gov%2Fresource%2Fdfd1938a-c890-442b-8341-c59d8fe6ee4d%2Fdownload
Reusing existing connection to edx.netl.doe.gov:443.
HTTP request sent, awaiting response... 200 OK
Length: 5686 (5.6K) [text/html]
Saving to: ‘matric_demo_sidcaad2ce9d34900c599a16c2d8b2ba8f1_discoveries_parsed_csv_21.zip’

matric_demo_sidcaad 100%[===================>]   5.55K  --.-KB/s    in 0s      

2023-03-29 13:01:14 (122

In [ ]:
### load the data

data = []

zf = zipfile.ZipFile('matric_demo_sidcaad2ce9d34900c599a16c2d8b2ba8f1_discoveries_parsed_csv_21.zip')

for f in zf.infolist():
        try: 
                zf.open(f.filename)
                df = pd.read_csv(zf.open(f.filename), usecols = ["body"], header=0)
                data.append(df)
        except:
                continue

# total length of list, this number equals total number of products
#print(len(data))

# first row of the list
#print(data[0])

discoveries = pd.concat(data, axis=0, ignore_index=True)
    


In [ ]:
discoveries.head()

In [ ]:
discoveries.isnull().sum()

In [ ]:
discoveries.body = discoveries.body.astype('str')

In [ ]:
discoveries.info()

In [ ]:
import re
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download('punkt')

from nltk.corpus import stopwords
stop_words = stopwords.words("english")
additional_stop_words = ["rt","like","amp","i'm","get","u","got","lol"] 
stop_words += additional_stop_words

from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

from nltk import word_tokenize
from nltk.util import ngrams

def preprocess(text):
  # Make all text lower case
  text = text.lower()
 
  #Removing special character
  text = re.sub('[!"”$%&’()*+,-./:;<=>?[\]^_`{|}~@#]','',text)
  
  #text = [word for word in text if len(word)>2]

  # Tokenize text
  text = word_tokenize(text)
  
  # Lemmatize text
  text = [lemmatizer.lemmatize(word) for word in text] 
  
  # Remove stopwords from text
  text = [word for word in text if word not in stop_words]
  return text

In [ ]:
from IPython.display import display
import collections

import itertools

def preprocess_ngrams(text, index):
    from nltk.util import ngrams
    text = text.lower()
    text = re.sub('[!"”$%&’()*+,-./:;<=>?[\]^_`{|}~@#]','',text)
    text = word_tokenize(text)
    terms_ngrams = [list(ngrams(text, index))]

    terms_ngrams = [ngram for ngram in terms_ngrams if ngram[0] not in stop_words and ngram[index-1] not in stop_words]
   
    grams = list(itertools.chain(*terms_ngrams))

    # ngram_counts = collections.Counter(grams)
    # ngram_counts.most_common(10)
    # ngram_df = pd.DataFrame(ngram_counts.most_common(10))
    return grams
    # print("\nNgram results: ")f
    # display(ngram_df.head())


In [ ]:
discoveries.head()

In [ ]:
#discoveries["clean_body"] = discoveries.body.apply(preprocess)

In [ ]:
# This is freezing before completing one iteration
n = 2
for i in range(1, n+1):
    discoveries[str(i) + "-grams"] = discoveries.body.apply(preprocess_ngrams,index = i)
    print(i)

In [ ]:
discoveries.head()

In [ ]:
df_columns = ['overall', 'body']
# if n>=1:
#     df_columns.append("clean_body")
# if n>=2:
#     df_columns.append("digrams")
# if n>=3:
#     df_columns.append("trigrams")
# if n>=4:
for i in range(1, n+1):
    df_columns.append(str(i) + "-grams")


In [ ]:
df = pd.DataFrame(discoveries, columns=['overall', 'body', 'clean_body'])


In [ ]:
word_count = dict()

for text in df['clean_body']:
  for word in text:
    if word in word_count: 
      word_count[word] += 1
    else:
      word_count[word] = 1

sorted(word_count.items(),key = lambda x: x[1],reverse=True)[:10]

In [ ]:
!pip install -qq pyLDAvis
!pip install -qq gensim

In [ ]:
from gensim.corpora.dictionary import Dictionary 
from gensim.models import LdaMulticore
import pyLDAvis.gensim

In [ ]:
dictionary = Dictionary(discoveries['clean_body'])

In [ ]:
# filter out low-frequency / high-frequency, also limit the vocabulary to max 1000 words
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in discoveries['clean_body']]

In [ ]:
#Train the model
lda = LdaMulticore(corpus, id2word=dictionary, num_topics=4, workers=3, passes=1)

In [ ]:
# Inspect the topics
lda.print_topics(0)

In [ ]:
# Visualizing
lda_vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary) 
pyLDAvis.display(lda_vis)

In [ ]:
lda[corpus][0]

In [ ]:
sorted(lda[corpus][0])[0][0]

In [ ]:
discoveries['topic'] = [sorted(lda[corpus][text])[0][0] for text in range(len(discoveries['clean_body']))]

In [ ]:
discoveries.topic.value_counts()

In [ ]:
#Using Spacy's pre-trained model (english version):
!python -m spacy download en_core_web_md -qq

#The code snippet above installs the larger-than-standard en_core_web_md library, 
#which includes 20k unique vectors with 300 dimensions.
#source: https://spacy.io/models/en

In [ ]:
import en_core_web_md
nlp2 = en_core_web_md.load(disable=["tagger", "parser", "ner"])

In [ ]:
from spacy.tokens import Doc

doc1 = nlp2(discoveries.iloc[0]["body"])
doc2 = nlp2(discoveries.iloc[1]["body"])
doc_list = []
print(discoveries.iloc[0]["clean_body"])
for sent in discoveries["clean_body"]:
    doc = Doc(nlp2.vocab, sent)
    doc_list.append(nlp2(doc))
print(doc_list[0].similarity(doc_list[1]))
print(discoveries.iloc[0]["clean_body"])
print(doc2)
print(doc1.similarity(doc2))

In [ ]:
spacy_embedding_dic = {}

for l in tokens:
    for word in l:
        spacy_embedding_dic[word] = nlp2(word).vector

In [ ]:
spacy_embedding_dic['fl']

In [ ]:
s_embedding = pd.DataFrame.from_dict(spacy_embedding_dic)
s_embedding.head()

In [ ]:
import umap

In [ ]:
reducer = umap.UMAP()

In [ ]:
umap_spacy = reducer.fit_transform(s_embedding)
umap_spacy.shape

In [ ]:
mapper = umap.UMAP().fit(s_embedding)

In [ ]:
from sklearn.cluster import KMeans

# and instantiate it where we need to specify that we want it to create 3 clusters (three types of penguins)
clusterer = KMeans(n_clusters=4)
clusterer.fit(s_embedding) #we only fit here, since no data needs to be transformed

In [ ]:
sns.scatterplot(umap_spacy[:,0], umap_spacy[:,1], hue = clusterer.labels_)